In [2]:
!pip install wordcloud
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import fuzzywuzzy
import re
from collections import Counter

In [3]:
df = pd.read_csv("airplane_crashes_and_fatalities_since_1908.csv")

In [4]:
df.head(10)

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,07/12/1912,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...
3,09/09/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,14.0,0.0,The airship flew into a thunderstorm and encou...
4,10/17/1913,10:30,"Near Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,NaN,30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...
5,03/05/1915,01:00,"Tienen, Belgium",Military - German Navy,NaN,NaN,Zeppelin L-8 (airship),NaN,NaN,41.0,21.0,0.0,Crashed into trees while attempting to land af...
6,09/03/1915,15:20,"Off Cuxhaven, Germany",Military - German Navy,NaN,NaN,Zeppelin L-10 (airship),NaN,NaN,19.0,19.0,0.0,"Exploded and burned near Neuwerk Island, when..."
7,07/28/1916,NaN,"Near Jambol, Bulgeria",Military - German Army,NaN,NaN,Schutte-Lanz S-L-10 (airship),NaN,NaN,20.0,20.0,0.0,"Crashed near the Black Sea, cause unknown."
8,09/24/1916,01:00,"Billericay, England",Military - German Navy,NaN,NaN,Zeppelin L-32 (airship),NaN,NaN,22.0,22.0,0.0,Shot down by British aircraft crashing in flames.
9,10/01/1916,23:45,"Potters Bar, England",Military - German Navy,NaN,NaN,Zeppelin L-31 (airship),NaN,NaN,19.0,19.0,0.0,Shot down in flames by the British 39th Home D...


In [5]:
df.shape

(5268, 13)

In [6]:
df.dtypes

Date             object
Time             object
Location         object
Operator         object
Flight #         object
Route            object
Type             object
Registration     object
cn/In            object
Aboard          float64
Fatalities      float64
Ground          float64
Summary          object
dtype: object

In [7]:
#Clean up time 
df["Time"] = df["Time"].replace(np.nan, "Unknown") 
df["Time"] = df["Time"].str.replace("114:20:00", "Unknown")
df["Time"] = df["Time"].str.replace("18.40", "18:40")
df["Time"] = df["Time"].str.replace("0943", "09:43")
df["Time"] = df["Time"].str.replace("12\'20", "12:20")
df["Time"] = df["Time"].str.replace("22\'08", "22:08")
df["Time"] = df["Time"].str.replace("c: ", "")
df["Time"] = df["Time"].str.replace("c:", "")
df["Time"] = df["Time"].str.replace("c", "")

#Match upper/lower case formats
df = df.applymap(lambda s:s.upper() if type(s) == str else s)

#Create new columns for month, day, year, hour, and minute
    
df['Date']=pd.to_datetime(df['Date'])
df['Day'] = df['Date'].map(lambda x: x.day)
df['Year'] = df['Date'].map(lambda x: x.year)
df['Month'] = df['Date'].map(lambda x: x.month)

df

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary,Day,Year,Month
0,1908-09-17,17:18,"FORT MYER, VIRGINIA",MILITARY - U.S. ARMY,NaN,DEMONSTRATION,WRIGHT FLYER III,NaN,1,2.0,1.0,0.0,"DURING A DEMONSTRATION FLIGHT, A U.S. ARMY FLY...",17,1908,9
1,1912-07-12,06:30,"ATLANTICITY, NEW JERSEY",MILITARY - U.S. NAVY,NaN,TEST FLIGHT,DIRIGIBLE,NaN,NaN,5.0,5.0,0.0,FIRST U.S. DIRIGIBLE AKRON EXPLODED JUST OFFSH...,12,1912,7
2,1913-08-06,UNKNOWN,"VICTORIA, BRITISH COLUMBIA, CANADA",PRIVATE,-,NaN,CURTISS SEAPLANE,NaN,NaN,1.0,1.0,0.0,THE FIRST FATAL AIRPLANE ACCIDENT IN CANADA OC...,6,1913,8
3,1913-09-09,18:30,OVER THE NORTH SEA,MILITARY - GERMAN NAVY,NaN,NaN,ZEPPELIN L-1 (AIRSHIP),NaN,NaN,20.0,14.0,0.0,THE AIRSHIP FLEW INTO A THUNDERSTORM AND ENCOU...,9,1913,9
4,1913-10-17,10:30,"NEAR JOHANNISTHAL, GERMANY",MILITARY - GERMAN NAVY,NaN,NaN,ZEPPELIN L-2 (AIRSHIP),NaN,NaN,30.0,30.0,0.0,HYDROGEN GAS WHICH WAS BEING VENTED WAS SUCKED...,17,1913,10
5,1915-03-05,01:00,"TIENEN, BELGIUM",MILITARY - GERMAN NAVY,NaN,NaN,ZEPPELIN L-8 (AIRSHIP),NaN,NaN,41.0,21.0,0.0,CRASHED INTO TREES WHILE ATTEMPTING TO LAND AF...,5,1915,3
6,1915-09-03,15:20,"OFF CUXHAVEN, GERMANY",MILITARY - GERMAN NAVY,NaN,NaN,ZEPPELIN L-10 (AIRSHIP),NaN,NaN,19.0,19.0,0.0,"EXPLODED AND BURNED NEAR NEUWERK ISLAND, WHEN...",3,1915,9
7,1916-07-28,UNKNOWN,"NEAR JAMBOL, BULGERIA",MILITARY - GERMAN ARMY,NaN,NaN,SCHUTTE-LANZ S-L-10 (AIRSHIP),NaN,NaN,20.0,20.0,0.0,"CRASHED NEAR THE BLACK SEA, CAUSE UNKNOWN.",28,1916,7
8,1916-09-24,01:00,"BILLERICAY, ENGLAND",MILITARY - GERMAN NAVY,NaN,NaN,ZEPPELIN L-32 (AIRSHIP),NaN,NaN,22.0,22.0,0.0,SHOT DOWN BY BRITISH AIRCRAFT CRASHING IN FLAMES.,24,1916,9
9,1916-10-01,23:45,"POTTERS BAR, ENGLAND",MILITARY - GERMAN NAVY,NaN,NaN,ZEPPELIN L-31 (AIRSHIP),NaN,NaN,19.0,19.0,0.0,SHOT DOWN IN FLAMES BY THE BRITISH 39TH HOME D...,1,1916,10


In [8]:
df1 = df[['Date', 'Operator', 'Aboard', 'Fatalities', 'Summary']].dropna(subset=['Aboard', 'Fatalities'])
df_nan = df1[pd.isnull(df1['Summary'])]
df_nan.head(10)

,Date,Operator,Aboard,Fatalities,Summary
21,1918-12-16,US AERIAL MAIL SERVICE,1.0,1.0,NaN
23,1919-07-19,US AERIAL MAIL SERVICE,1.0,1.0,NaN
24,1919-10-02,AIRCRAFT TRANSPORT AND TRAVEL,1.0,1.0,NaN
25,1919-10-14,US AERIAL MAIL SERVICE,1.0,1.0,NaN
27,1919-10-30,US AERIAL MAIL SERVICE,1.0,1.0,NaN
28,1920-03-10,US AERIAL MAIL SERVICE,1.0,1.0,NaN
29,1920-03-30,US AERIAL MAIL SERVICE,1.0,1.0,NaN
30,1920-04-10,US AERIAL MAIL SERVICE,2.0,1.0,NaN
31,1920-04-11,US AERIAL MAIL SERVICE,2.0,1.0,NaN
32,1920-05-12,US AERIAL MAIL SERVICE,2.0,1.0,NaN


In [9]:
df1 = df[['Date', 'Operator', 'Aboard', 'Fatalities', 'Summary']].dropna(subset=['Aboard', 'Fatalities', 'Summary'])
df_unknown = df1[df1["Summary"].str.contains("UNKNOWN")]
df_unknown

,Date,Operator,Aboard,Fatalities,Summary
7,1916-07-28,MILITARY - GERMAN ARMY,20.0,20.0,"CRASHED NEAR THE BLACK SEA, CAUSE UNKNOWN."
362,1935-07-19,WEDELL-WILLIAMS AIR SERVICE CORPORATION,2.0,2.0,THE PLANE DEVIATED FROM ITS COURSE AND FLEW 20...
390,1936-04-07,TRANS CONTINENTAL AND WESTERN AIR,15.0,13.0,CRASHED INTO CHESTNUT RIDGE IN FOG. THE POOR ...
403,1936-08-05,CHICAGO SOUTHERN AIRLINES,8.0,8.0,CRASHED WHILE ATTEMPTING TO TAKE OFF FROM LAMB...
480,1938-07-29,PAN AMERICAN WORLD AIRWAYS,15.0,15.0,THE PLANE CRASHED INTO THE OCEAN WHILE EN ROUT...
566,1941-10-30,AMERICAN AIRLINES,20.0,20.0,"THE AIRCRAFT, ON A FLIGHT FROM NEW YORK TO CHI..."
568,1942-01-16,TRANS CONTINENTAL AND WESTERN AIR,22.0,22.0,THE AIRCRAFT CRASHED INTO MT. POTOSI (DOUBLE U...
591,1942-08-25,MILITARY - ROYAL AIR FORCE,14.0,13.0,THE PLANE CRASHED WHILE ON A FLIGHT TO ICELAND...
620,1943-07-28,AMERICAN AIRLINES,22.0,20.0,THE AIRCRAFT CRASHED INTO TREES WHILE ATTEMPTI...
667,1944-09-18,U. S. AIR FORCE,19.0,19.0,STRUCK THE SIDE OF A MOUNTAIN WHILE IN FULL FL...


In [10]:
df1 = df[['Date', 'Operator', 'Aboard', 'Fatalities', 'Summary']].dropna(subset=['Aboard', 'Fatalities', 'Summary'])
df_weather = df1[df1["Summary"].str.contains("VISIBILITY")]
df_weather

,Date,Operator,Aboard,Fatalities,Summary
60,1922-04-07,DAIMLER AIRWAYS / GRANDS EXPRESS AERIENS,7.0,7.0,A MIDAIR COLLISION OCCURRED IN POOR VISIBILITY...
240,1931-05-24,NATIONAL AIR TRANSPORT,1.0,1.0,CRASHED INTO A MOUNTAIN AND BURNED IN POOR VIS...
241,1931-06-07,AIR ORIENT,4.0,4.0,CRASHED INTO A TREES AND FELL INTO A RIVER 50 ...
293,1933-01-20,AMERICAN AIRWAYS,1.0,1.0,THE PILOT OF THE MAIL PLANE LOST HIS BEARINGS ...
313,1933-12-11,DEUTSCHE LUFTHANSA,8.0,7.0,CRASHED AFTER STRIKING AN OBSTACLE WHILE LANDI...
335,1934-10-03,QANTAS,3.0,3.0,"CRASHED AFTER IN-FLIGHT LOSS OF CONTROL, POSSI..."
345,1934-12-22,AMERICAN AIRLINES,1.0,1.0,"CRASHED INTO BIG PILOT MOUNTAIN, 50 FEET FROM ..."
354,1935-05-06,TRANS CONTINENTAL AND WESTERN AIR,14.0,5.0,THE PLANE CRASHED WHILE EN ROUTE FROM ALBUQUER...
390,1936-04-07,TRANS CONTINENTAL AND WESTERN AIR,15.0,13.0,CRASHED INTO CHESTNUT RIDGE IN FOG. THE POOR ...
424,1936-12-27,UNITED AIR LINES,12.0,12.0,"WHILE ON APPROACH TO BURBANK AIRPORT, THE AIRL..."


In [11]:
df1 = df[['Date', 'Operator', 'Aboard', 'Fatalities', 'Summary']].dropna(subset=['Aboard', 'Fatalities', 'Summary'])
df_weather = df1[df1["Summary"].str.contains("FOG")]
df_weather

,Date,Operator,Aboard,Fatalities,Summary
43,1920-12-14,HANDLEY PAGE TRANSPORT,8.0,4.0,CRASHED INTO A TREE ON TAKE OFF IN FOG.
132,1928-01-08,COLONIAL WESTERN AIRLINES,3.0,3.0,BECAME LOST IN DENSE FOG AND FLEW INTO THE GRO...
141,1928-04-19,IBERIA AIRLINES,9.0,9.0,CRASHED IN FOG.
150,1928-08-25,BRITISH COLUMBIA AIRWAYS,7.0,7.0,CRASHED INTO PUGET SOUND IN DENSE FOG. CONSID...
168,1928-12-31,CIDNA,3.0,2.0,CRASHED INTO HIGH GROUND IN FOG.
175,1929-03-30,NEW STANDARD AIRLINES,4.0,4.0,THE AIRCRAFT CRASHED INTO A HIGH MOUNTAIN RIDG...
198,1929-11-04,CORP. AERONAUTICA DE TRANSPORTES,4.0,4.0,THE MAIL PLANE CRASHED IN DENSE FOG.
199,1929-11-06,DEUTSCHE LUFTHANSA,8.0,7.0,CRASHED IN HEAVY RAIN AND FOG WHILE ATTEMPTING...
200,1929-11-15,AIR UNION,3.0,3.0,CRASHED INTO THE MEDITERRANEAN IN FOG.
201,1929-11-30,VARNEY AIR LINES,1.0,1.0,THE MAIL PLANE CRASHED INTO A FOG BOUND TOWER ...


In [12]:
df1 = df[['Date', 'Operator', 'Aboard', 'Fatalities', 'Summary']].dropna(subset=['Aboard', 'Fatalities', 'Summary'])
df_weather = df1[df1["Summary"].str.contains("SEA")]
df_weather

,Date,Operator,Aboard,Fatalities,Summary
3,1913-09-09,MILITARY - GERMAN NAVY,20.0,14.0,THE AIRSHIP FLEW INTO A THUNDERSTORM AND ENCOU...
7,1916-07-28,MILITARY - GERMAN ARMY,20.0,20.0,"CRASHED NEAR THE BLACK SEA, CAUSE UNKNOWN."
11,1916-11-28,MILITARY - GERMAN NAVY,20.0,20.0,SHOT DOWN BY BRITISH ANTI-AIRCRAFT FIRE AND AI...
13,1917-03-30,MILITARY - GERMAN NAVY,23.0,23.0,STRUCK BY LIGHTNING AND CRASHED INTO THE BALTI...
14,1917-05-14,MILITARY - GERMAN NAVY,21.0,21.0,"CRASHED INTO THE SEA FROM AN ALTITUDE OF 3,000..."
18,1918-04-07,MILITARY - GERMAN NAVY,23.0,23.0,EXPLODED AND CRASHED INTO THE SEA OFF THE SOUT...
108,1926-10-21,IMPERIAL AIRWAYS,12.0,0.0,THE PLANE WAS ON A FLIGHT FROM LONDON TO PARIS...
163,1928-12-03,SYNDICATO CONDOR,14.0,14.0,THE SEAPLANE TOOK OFF FORM THE BAY AND NARROWL...
227,1930-11-07,SYNDICATO CONDOR,9.0,1.0,CRASHED INTO THE SEA.
336,1934-10-19,HOLYMAN AIRWAYS,11.0,11.0,CRASHED EN ROUTE DURING THE 265 MILES SEA ROUTE.


In [13]:
df1 = df[['Date', 'Operator', 'Aboard', 'Fatalities', 'Summary']].dropna(subset=['Aboard', 'Fatalities', 'Summary'])
df_weather = df1[df1["Summary"].str.contains("LAND")]
df_weather

,Date,Operator,Aboard,Fatalities,Summary
3,1913-09-09,MILITARY - GERMAN NAVY,20.0,14.0,THE AIRSHIP FLEW INTO A THUNDERSTORM AND ENCOU...
5,1915-03-05,MILITARY - GERMAN NAVY,41.0,21.0,CRASHED INTO TREES WHILE ATTEMPTING TO LAND AF...
6,1915-09-03,MILITARY - GERMAN NAVY,19.0,19.0,"EXPLODED AND BURNED NEAR NEUWERK ISLAND, WHEN..."
40,1920-10-05,LATECOERE AIRLINES,2.0,2.0,CRASHED WHILE LANDING.
60,1922-04-07,DAIMLER AIRWAYS / GRANDS EXPRESS AERIENS,7.0,7.0,A MIDAIR COLLISION OCCURRED IN POOR VISIBILITY...
88,1924-12-24,IMPERIAL AIRWAYS,8.0,8.0,"SHORTLY AFTER TAKING OFF, WITNESSES OBSERVED A..."
106,1926-09-26,NaN,5.0,2.0,TOO MUCH WEIGHT CAUSED THE LANDING GEAR TO COL...
113,1927-03-24,QANTAS,3.0,3.0,STALLED AT LOW ALTITIUDE ON APPROACH TO LAND
119,1927-08-22,KLM ROYAL DUTCH AIRLINES,11.0,1.0,A BROKEN WIRE LED TO THE PARTIAL LOSS OF THE T...
170,1929-01-18,VARNEY AIR LINES,1.0,1.0,WHILE ATTEMPTING TO MAKE AN EMERGENCY LANDING ...


# Esketit

In [14]:
df1 = df[['Year', 'Operator', 'Aboard', 'Fatalities', 'Summary']].dropna(subset=['Operator', 'Aboard', 'Fatalities', 'Summary'])
df1.head(100)

,Year,Operator,Aboard,Fatalities,Summary
0,1908,MILITARY - U.S. ARMY,2.0,1.0,"DURING A DEMONSTRATION FLIGHT, A U.S. ARMY FLY..."
1,1912,MILITARY - U.S. NAVY,5.0,5.0,FIRST U.S. DIRIGIBLE AKRON EXPLODED JUST OFFSH...
2,1913,PRIVATE,1.0,1.0,THE FIRST FATAL AIRPLANE ACCIDENT IN CANADA OC...
3,1913,MILITARY - GERMAN NAVY,20.0,14.0,THE AIRSHIP FLEW INTO A THUNDERSTORM AND ENCOU...
4,1913,MILITARY - GERMAN NAVY,30.0,30.0,HYDROGEN GAS WHICH WAS BEING VENTED WAS SUCKED...
5,1915,MILITARY - GERMAN NAVY,41.0,21.0,CRASHED INTO TREES WHILE ATTEMPTING TO LAND AF...
6,1915,MILITARY - GERMAN NAVY,19.0,19.0,"EXPLODED AND BURNED NEAR NEUWERK ISLAND, WHEN..."
7,1916,MILITARY - GERMAN ARMY,20.0,20.0,"CRASHED NEAR THE BLACK SEA, CAUSE UNKNOWN."
8,1916,MILITARY - GERMAN NAVY,22.0,22.0,SHOT DOWN BY BRITISH AIRCRAFT CRASHING IN FLAMES.
9,1916,MILITARY - GERMAN NAVY,19.0,19.0,SHOT DOWN IN FLAMES BY THE BRITISH 39TH HOME D...


Look for crashes in dataframe

In [15]:
df_military = df1[df1["Operator"].str.contains("MILITARY")]
df_military.head(10)

,Year,Operator,Aboard,Fatalities,Summary
0,1908,MILITARY - U.S. ARMY,2.0,1.0,"DURING A DEMONSTRATION FLIGHT, A U.S. ARMY FLY..."
1,1912,MILITARY - U.S. NAVY,5.0,5.0,FIRST U.S. DIRIGIBLE AKRON EXPLODED JUST OFFSH...
3,1913,MILITARY - GERMAN NAVY,20.0,14.0,THE AIRSHIP FLEW INTO A THUNDERSTORM AND ENCOU...
4,1913,MILITARY - GERMAN NAVY,30.0,30.0,HYDROGEN GAS WHICH WAS BEING VENTED WAS SUCKED...
5,1915,MILITARY - GERMAN NAVY,41.0,21.0,CRASHED INTO TREES WHILE ATTEMPTING TO LAND AF...
6,1915,MILITARY - GERMAN NAVY,19.0,19.0,"EXPLODED AND BURNED NEAR NEUWERK ISLAND, WHEN..."
7,1916,MILITARY - GERMAN ARMY,20.0,20.0,"CRASHED NEAR THE BLACK SEA, CAUSE UNKNOWN."
8,1916,MILITARY - GERMAN NAVY,22.0,22.0,SHOT DOWN BY BRITISH AIRCRAFT CRASHING IN FLAMES.
9,1916,MILITARY - GERMAN NAVY,19.0,19.0,SHOT DOWN IN FLAMES BY THE BRITISH 39TH HOME D...
10,1916,MILITARY - GERMAN ARMY,28.0,27.0,CRASHED IN A STORM.


In [16]:
year = df_military["Year"]
year.head(10)

0     1908
1     1912
3     1913
4     1913
5     1915
6     1915
7     1916
8     1916
9     1916
10    1916
Name: Year, dtype: int64

In [17]:
m_fatalities_count = df_military.groupby(["Year"])['Fatalities'].sum()
m_fatalities_count.head(5)

Year
1908      1.0
1912      5.0
1913     44.0
1915     40.0
1916    108.0
Name: Fatalities, dtype: float64

In [18]:
m_totals_count = df_military.groupby(["Year"])['Aboard'].sum()
m_totals_count.head(5)

Year
1908      2.0
1912      5.0
1913     50.0
1915     60.0
1916    109.0
Name: Aboard, dtype: float64

In [19]:
m_fatality_rate = fatalities_count / totals_count
m_fatality_rate.head(20)

NameError: name 'fatalities_count' is not defined

In [ ]:
df2_military = pd.DataFrame({
    "Fatalities": m_fatalities_count,
    "Total Passengers": m_totals_count,
    "Fatality Rate": m_fatality_rate})

df2_military.head(15)

In [ ]:
df2_military = pd.DataFrame({
    "Year": year,
    "Fatality Rate": m_fatality_rate})

df2_military = df2_military.dropna(subset=['Year', 'Fatality Rate'])
df2_military

In [ ]:
military_chart = df2_military.plot(kind="line")
plt.show()

In [ ]:
df_crash = df1[df1["Summary"].str.contains("CRASHED", "CRASH")]

In [ ]:
df_commercial = df1[df1["Operator"].str.contains("MILITARY") == False]
df_commercial.head(10)

In [ ]:
c_fatalities_count = df_commercial.groupby(["Year"])['Fatalities'].sum()
c_fatalities_count.head(15)

In [ ]:
c_totals_count = df_commercial.groupby(["Year"])['Aboard'].sum()
c_totals_count.head(15)

In [ ]:
c_fatality_rate = fatalities_count / totals_count
c_fatality_rate.head(15)

In [ ]:
df2_commercial = pd.DataFrame({
    "Fatalities": c_fatalities_count,
    "Total Passengers": c_totals_count,
    "Fatality Rate": c_fatality_rate})

df2_commercial.head(15)

In [ ]:
df2_commercial = pd.DataFrame({

    "Fatality Rate": c_fatality_rate})

df2_commercial.head(15)

In [ ]:
commercial_chart = df2_commercial.plot(kind = "line")
plt.show()

In [20]:
commercial_aboard = df_commercial["Aboard"].sum()
commercial_aboard

NameError: name 'df_commercial' is not defined

In [21]:
commercial_fatalities = df_commercial["Fatalities"].sum()
commercial_fatalities

NameError: name 'df_commercial' is not defined

In [22]:
military_fatalities/military_aboard

NameError: name 'military_fatalities' is not defined

In [23]:
commercial_fatalities/commercial_aboard

NameError: name 'commercial_fatalities' is not defined

In [24]:
df_commercial["Result"] = df_commercial["Fatalities"] / df_commercial["Aboard"]
df_commercial

NameError: name 'df_commercial' is not defined

In [25]:
df_commercial.plot(x="Date", y="Result")
plt.ylabel("Fatality Rate")

NameError: name 'df_commercial' is not defined

In [26]:
df_military["Fatality Rate"] = df_military["Fatalities"] / df_military["Aboard"]
df_military

C:\Users\ericl\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Year,Operator,Aboard,Fatalities,Summary,Fatality Rate
0,1908,MILITARY - U.S. ARMY,2.0,1.0,"DURING A DEMONSTRATION FLIGHT, A U.S. ARMY FLY...",0.500000
1,1912,MILITARY - U.S. NAVY,5.0,5.0,FIRST U.S. DIRIGIBLE AKRON EXPLODED JUST OFFSH...,1.000000
3,1913,MILITARY - GERMAN NAVY,20.0,14.0,THE AIRSHIP FLEW INTO A THUNDERSTORM AND ENCOU...,0.700000
4,1913,MILITARY - GERMAN NAVY,30.0,30.0,HYDROGEN GAS WHICH WAS BEING VENTED WAS SUCKED...,1.000000
5,1915,MILITARY - GERMAN NAVY,41.0,21.0,CRASHED INTO TREES WHILE ATTEMPTING TO LAND AF...,0.512195
6,1915,MILITARY - GERMAN NAVY,19.0,19.0,"EXPLODED AND BURNED NEAR NEUWERK ISLAND, WHEN...",1.000000
7,1916,MILITARY - GERMAN ARMY,20.0,20.0,"CRASHED NEAR THE BLACK SEA, CAUSE UNKNOWN.",1.000000
8,1916,MILITARY - GERMAN NAVY,22.0,22.0,SHOT DOWN BY BRITISH AIRCRAFT CRASHING IN FLAMES.,1.000000
9,1916,MILITARY - GERMAN NAVY,19.0,19.0,SHOT DOWN IN FLAMES BY THE BRITISH 39TH HOME D...,1.000000
10,1916,MILITARY - GERMAN ARMY,28.0,27.0,CRASHED IN A STORM.,0.964286


In [27]:
df_military.plot(x="Date", y="Fatality Rate")

KeyError: 'Date'

In [ ]:
df_military["Result"] = df_military["Aboard"] - df_military["Fatalities"] 
df_military

In [ ]:
df_military.plot(x="Date", y="Result")